In [ ]:
cd /home/yuchen/pulse2percept

In [ ]:
import shapes
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import pulse2percept as p2p
import pandas as pd
import skimage
from skimage.measure import label, regionprops, regionprops_table
import math
import string
from statistics import mean
from sklearn.linear_model import LinearRegression
import shapely.geometry as geom

In [ ]:
# calculate shapes 
data = shapes.load_shapes("/home/yuchen/shapes/data/shapes.h5", subjects=['12-005','51-009','52-001'],stim_class=None)

result = pd.DataFrame({})
for i in range(len(data)):
    label_img = skimage.measure.label(data['image'][i]>0)
    regions = regionprops(label_img)
    props = regionprops_table(label_img, properties=('centroid',
                                                     'orientation',
                                                     'major_axis_length',
                                                     'minor_axis_length',
                                                     'area',
                                                    'eccentricity',
                                                    'perimeter'))
    df = pd.DataFrame(props).astype('object')
    df.at[0,'centroid-0'] = df.iloc[:, 0].tolist()  # store centroid-x
    df.at[0,'centroid-1'] = df.iloc[:, 1].tolist()  # store centroid-y
    df.at[0,'orientation'] = df.iloc[:, 2].tolist()  # store orientation
    df.at[0,'major_axis_length'] = df.iloc[:, 3].tolist()  # major
    df.at[0,'minor_axis_length'] = df.iloc[:, 4].tolist()  # minor
    df.at[0,'area'] = df.iloc[:, 5].tolist()  # area
    df.at[0,'eccentricity'] = df.iloc[:, 6].tolist()  
    df.at[0,'perimeter'] = df.iloc[:, 7].tolist()  
    result = pd.concat([result, df.iloc[:1,:]],axis=0)

result = result.rename(columns={"area":"size", "orientation":"orientation_new", "eccentricity":"eccentricity_new" })
data = pd.concat([data,result.reset_index(drop=True)],axis=1)
data_copy = data 

In [ ]:
count_0 = 0
count_1 = 0
test = []
lst = []
for i in range(len(data)):
    if data.electrode2[i] == str():
        lst.append('SingleElectrode')
    else:
        lst.append(data.stim_class[i])
data['stim_class'] = lst

cate = data[['subject','amp1','freq','electrode1','electrode2','stim_class']].drop_duplicates().reset_index(drop=True)

img = []
for i in range(len(cate)):
    df = data[(data['subject'] == cate['subject'][i]) & 
              (data['amp1'] == cate['amp1'][i]) & 
              (data['electrode1'] == cate['electrode1'][i]) &
              (data['electrode2'] == cate['electrode2'][i]) & 
              (data['stim_class'] == cate['stim_class'][i])
             ].reset_index(drop=True)
    if df.empty:
        img.append(np.zeros((384, 512)))
    else:
        if df.electrode2[0] == str():  # single electrode 
            if (len(df[df['num_regions']>1])/(len(df))) <= 0.5:
                count_1 += 1
                df_1 = df[df['num_regions'] == 1].reset_index(drop=True)
                df_centroid_0_avg = 0
                df_centroid_1_avg = 0
                for item in range(len(df_1)):
                    df_centroid_0_avg = df_centroid_0_avg + df_1['centroid-0'][item][0]
                    df_centroid_1_avg = df_centroid_1_avg + df_1['centroid-1'][item][0]
                df_centroid_0_avg = df_centroid_0_avg/len(df_1)
                df_centroid_1_avg = df_centroid_1_avg/len(df_1)

                # for drawings with more than one phosphenes, select the one with centroid cloest to the averaged centroid
                df_2 = df[df['num_regions'] > 1].reset_index(drop=True)
                if len(df_2) != 0:
                    for double_number in range(len(df_2)):
                        phosphene_id = 0
                        distance = 9999999
                        for double_item in range(len(df_2['centroid-0'][double_number])):
                            distance_temp = (df_2['centroid-0'][double_number][double_item] - df_centroid_0_avg)**2 + (df_2['centroid-1'][double_number][double_item] - df_centroid_1_avg)**2
                            if distance_temp < distance:
                                distance = distance_temp
                                phosphene_id = double_item
                        df_2.at[double_number,'centroid-0'] = [df_2['centroid-0'][double_number][phosphene_id]]
                        df_2.at[double_number,'centroid-1'] = [df_2['centroid-1'][double_number][phosphene_id]]

                df = pd.concat([df_1,df_2], axis=0).reset_index(drop=True)
                image_temp = np.zeros((len(df['image'][0]),len(df['image'][0][0])))
                for image in range(len(df)):
                    image_temp = image_temp + p2p.utils.images.shift_image(df['image'][image],df_centroid_1_avg-df['centroid-1'][image][0], df_centroid_0_avg-df['centroid-0'][image][0])
                image_temp = image_temp / len(df)
                img.append(image_temp)

            else:  # remove single electrode data with >=50% drawings having >1 phoephenes
                img.append(np.zeros((len(df['image'][0]),len(df['image'][0][0]))))
                count_0 += 1 
        elif df.electrode2[0] != str():  # double electrode
            phosphene1 = np.mean([p2p.utils.images.center_image(skimage.measure.label(image) == 1) for image in df.image], axis=0)
            phosphene2 = np.mean([p2p.utils.images.center_image(skimage.measure.label(image) == 2) for image in df.image], axis=0)
            phosphene1_centroid = np.mean([regionprops(skimage.measure.label(image>0))[0].centroid for image in df.image], axis=0)
            if sum(sum(phosphene2)) != 0:
                count = (0,0)
                phosphene2_centroid = 0
                for image in df.image:
                    if len(regionprops(skimage.measure.label(image>0))) >= 2:
                        count = (count[0] + regionprops(skimage.measure.label(image>0))[1].centroid[0],count[1] + regionprops(skimage.measure.label(image>0))[1].centroid[1]) 
                        phosphene2_centroid += 1
                phosphene2_centroid = (count[0] / phosphene2_centroid,count[1] / phosphene2_centroid)
            y_center, x_center = phosphene1.shape[0] / 2, phosphene1.shape[1] / 2
            phosphene1 = p2p.utils.images.shift_image(phosphene1, phosphene1_centroid[1] - x_center, phosphene1_centroid[0] - y_center)
            if sum(sum(phosphene2)) != 0:
                phosphene2 = p2p.utils.images.shift_image(phosphene2, phosphene2_centroid[1] - x_center, phosphene2_centroid[0] - y_center)
                image_temp = phosphene1 + phosphene2
            else:
                image_temp = phosphene1
            test.append(image_temp)
            img.append(image_temp)
            
cate['avg_image'] = img

In [ ]:
cate = cate[cate.stim_class != 'SpatialSummation'].reset_index(drop=True)
cate

In [ ]:
temp = cate[cate.electrode2 != str()].reset_index(drop=True)
# temp = temp[(temp['subject'] == '52-001') 
#             &  (temp['electrode1'] == 'C10') & (temp['electrode2'] == 'F10')].reset_index(drop=True)
for i in range(len(temp)):
    f1 = cate[(cate.electrode1 == temp.electrode1[i]) & (cate.electrode2 == str()) & (cate.amp1 == temp.amp1[i]) & (cate.freq == temp.freq[i]) & (cate.subject == temp.subject[i])]
    f2 = cate[(cate.electrode1 == temp.electrode2[i]) & (cate.electrode2 == str()) & (cate.amp1 == temp.amp1[i]) & (cate.freq == temp.freq[i]) & (cate.subject == temp.subject[i])]
    
    if not f1.empty and not f2.empty:
        ig, axs= plt.subplots(ncols=3, nrows = 1, figsize=(42, 14))
        axs[0].imshow(f1.avg_image.iloc[0],cmap = 'gray')
        axs[1].imshow(f2.avg_image.iloc[0],cmap = 'gray')       
        axs[2].imshow(temp.avg_image[i],cmap = 'gray')
        axs[0].set(title=f1.electrode1.iloc[0] + ' ' + f1.electrode2.iloc[0])
        axs[1].set(title=f2.electrode1.iloc[0] + ' ' + f2.electrode2.iloc[0])
        axs[2].set(title=temp.electrode1[i] + ' ' + temp.electrode2[i] + ' ' + temp.subject[i])
#         for j in range(3):
#             axs[j].axis('off')

In [ ]:
temp = cate[cate.electrode2 != str()].reset_index(drop=True)
temp = temp[(temp['subject'] == '12-005') 
            &  (temp['electrode1'] == 'A1') & (temp['electrode2'] == 'A6')].reset_index(drop=True)